# Create MoBIE HTM Project

Create a MoBIE project for high-throughput-microscopy data. The test data for this example is available here: https://owncloud.gwdg.de/index.php/s/eu8JMlUFZ82ccHT. It contains a few wells  of a plate from a immunofluorescence based SARS-CoV-2 antibody assay from https://onlinelibrary.wiley.com/doi/full/10.1002/bies.202000257.

In [ ]:
# general imports
import os
from glob import glob
import mobie.htm as htm

# the location of the data
# adapt these paths to your system and the input data you are using

# location of the input data. 
# the example data used in this notebook is available via this link:
# https://oc.embl.de/index.php/s/IV1709ZlcUB1k99
example_input_folder = "/home/pape/Work/data/htm-test-data"

# the location of the mobie project that will be created
# we recommend that the mobie project folders have the structure <PROECJT_ROOT_FOLDER/data>
# the folder 'data' will contain the sub-folders for individual datasets
mobie_project_folder = "/home/pape/Work/data/mobie/mobie_htm_project/data"

# name of the dataset that will be created.
# one project can contain multiple datasets
dataset_name = "example-dataset"
dataset_folder = os.path.join(mobie_project_folder, dataset_name)

# the platform and number of jobs used for computation.
# choose 'local' to run computations on your machine.
# for large data, it is also possible to run computation on a cluster;
# for this purpose 'slurm' (for slurm cluster) and 'lsf' (for lsf cluster) are currently supported
target = "local"
max_jobs = 4

## Adding image data

In [ ]:
input_files = glob(os.path.join(example_input_folder, "*.h5"))
input_files.sort()

resolution = [0.65, 0.65]
scale_factors = 4 * [[2, 2]]
chunks = [512, 512]

channels = ["serum", "marker", "nuclei"]
for channel_name in channels:
    image_names = [os.path.splitext(os.path.basename(im))[0] for im in input_files]
    image_names = [f"{name}-{channel_name}" for name in image_names]

    htm.add_images(input_files, mobie_project_folder, dataset_name,
                   image_names, resolution, scale_factors, chunks, key=channel_name,
                   target=target, max_jobs=max_jobs, file_format="ome.zarr")

## Add segmentation data

In [ ]:
segmentation_names = ["cells", "nuclei"]
for seg_name in segmentation_names:
    image_names = [os.path.splitext(os.path.basename(im))[0] for im in input_files]
    image_names = [f"{name}-{seg_name}-segmentation" for name in image_names]
    
    htm.add_segmentations(input_files, mobie_project_folder, dataset_name,
                          image_names, resolution, scale_factors, chunks, key=f"segmentation/{seg_name}",
                          target=target, max_jobs=max_jobs, file_format="ome.zarr")

## Add views to creae plate layout

In [ ]:
# TODO